In [1]:
import torch
import numpy as np
from model_creator import make_conv2d_model, mod
device='cpu'
%load_ext autoreload
%autoreload 2


In [5]:
params = {}
params['n_conv_layers'] = 1
params['use_batch_norm'] = True
params['input_kernel_size'] = (5, 5)
params['conv_dim_change'] = 'double'
params['pool_size'] = (2, 2)
params['activation'] = 'relu'
params['n_dense_layers'] = 0
params['conv_kernel_size'] = (3, 3)
params['initial_kernel_number'] = 15
params['dense_dim'] = 100

In [6]:
model = mod(*make_conv2d_model((3, 32, 32), 0, params)).to(device)


In [11]:
out = model(torch.zeros((2, 3, 32, 32)))

In [19]:
model.lay[0].weight.shape

torch.Size([15, 3, 5, 5])